# 使用数据集工具
## 使用数据集工具
使用 huggingface 数据集工具加载数据集 已 chnSentiCorp 数据集为例，数据集详细介绍可以参考：https://ieee-dataport.org/open-access/chnsenticorp


In [ ]:
# pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset("lansinuote/ChnSentiCorp")
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9600 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})

### 1. 将数据保存到本地

In [ ]:
dataset.save_to_disk(dataset_dict_path='/home')

Saving the dataset (0/1 shards):   0%|          | 0/9600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1200 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1200 [00:00<?, ? examples/s]

### 2. 从磁盘加载数据集

In [ ]:
from datasets import load_from_disk
dataset = load_from_disk('/home/dataset/ChnSentiCorp')
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})


### 数据集基本操作
#### 1.取出数据部分

In [ ]:
datasetTrain = dataset['train']
print(datasetTrain)

Dataset({
    features: ['text', 'label'],
    num_rows: 9600
})


#### 2.查看数据内容

In [ ]:
datasetTrain[1], datasetTrain[200], datasetTrain[1000]

({'text': '15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错',
  'label': 1},
 {'text': '这本书虽然内容不多，但是插图很大，很生动和形象，里面的故事都是小孩子碰到过的事情，并且很有启发性，我女儿看了以后很喜欢，而且碰到术中类似的事情，她会说“大卫，不可以”，呵呵，看来是受到潜移默化的影响了，总的来说，个人感觉这本书不错，难怪是获过大奖的书。',
  'label': 0},
 {'text': '外壳容易印指纹，比较难看，贴上膜好一些，但膜太难贴了，气泡好多 整体有一点点重', 'label': 0})

#### 3. 数据排序利用 sort() 函数

In [ ]:
# 提取前10个数据
print(datasetTrain['label'][:10])
# 数据集排序
sorted_dataset = datasetTrain.sort('label')
print(sorted_dataset['label'][:10])
print(sorted_dataset['label'][-10:])

[1, 1, 0, 0, 1, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


#### 4. 打乱数据利用 shuffle() 函数

In [ ]:
shuffle_dataset = sorted_dataset.shuffle(seed=1000)
print(shuffle_dataset['label'][:10])
print(shuffle_dataset['label'][-10:])

[1, 0, 0, 0, 0, 1, 1, 0, 0, 1]
[0, 1, 0, 1, 0, 0, 1, 1, 1, 0]


#### 5. 数据抽样利用 select() 函数

In [ ]:
datasetTrain.select([0,10,20,30,40,50])

Dataset({
    features: ['text', 'label'],
    num_rows: 6
})

#### 6.数据过滤利用 filter() 函数

In [ ]:
# 定义过滤函数 filter() 函数是以函数为参数
def f(dataset):
  return dataset['text'].startswith('很好')

print(datasetTrain.filter(f))
# lambda 写法
print(datasetTrain.filter(lambda example: example['text'].startswith('不错')))

# 使用索引进行过滤
even_dataset = datasetTrain.filter(lambda example, idx: idx % 2 == 0, with_indices=True)
even_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 18
})
Dataset({
    features: ['text', 'label'],
    num_rows: 37
})


Dataset({
    features: ['text', 'label'],
    num_rows: 4800
})

#### 7. 训练集测试集拆分 利用 train_test_split()函数
其他重要参数
- shuffle 在分割数据集前是否打乱数据集
- seed 乱序随机种子
- keep_in_memory 将分割后的数据放入内存中，而不是缓存文件

In [ ]:
datasetTrain.train_test_split(test_size = 0.1) # 训练集和测试 9：1

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8640
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 960
    })
})

#### 8.数据分桶 利用 shard() 函数将数据均分成 n 部分
- num_shards 设置分成的分数
- index 取第几份

In [ ]:
datasetTrain.shard(num_shards = 4, index = 0)

Dataset({
    features: ['text', 'label'],
    num_rows: 2400
})

#### 9.重命名字段 利用函数 rename_column() 函数

In [ ]:
datasetTrain.rename_column('text','text_rename')

Dataset({
    features: ['text_rename', 'label'],
    num_rows: 9600
})

#### 10.删除字段 利用 remove_columns() 函数

In [ ]:
datasetTrain.remove_columns('text')

Dataset({
    features: ['label'],
    num_rows: 9600
})

#### 11.映射字段 利用 map() 函数遍历数据 参数为函数

In [45]:
# 调用函数写法
def f(data):
  data['text']= 'My sentence: ' + data['text']
  return data
mapDatasetTrain = datasetTrain.map(f)
print(datasetTrain['text'][20])
print(mapDatasetTrain['text'][20])

# lambda 写法
mapDatasetTrain1 = datasetTrain.map(lambda data: {**data, 'text': 'My sentence: ' + data['text']})
print(mapDatasetTrain1['text'][20])

非常不错，服务很好，位于市中心区，交通方便，不过价格也高！
My sentence: 非常不错，服务很好，位于市中心区，交通方便，不过价格也高！
My sentence: 非常不错，服务很好，位于市中心区，交通方便，不过价格也高！


#### 12. 使用批处理加速 利用 map() 函数进行批量处理任务，加速处理  
注意 使用 map() 函数进行批量处理是数据需要是列表而不是字符串

In [49]:
def f(data):
  text = data['text']
  text = ['My sentence: ' + i for i in text ]
  data['text'] = text
  return data
maped_DatasetTrain1 = datasetTrain.map(function=f,
                    batched = True,
                    batch_size = 1000,
                    num_proc = 4,
                    )
print(datasetTrain['text'][2])
print(maped_DatasetTrain1['text'][2])

# lambda 写法
maped_DatasetTrain2 = datasetTrain.map(function=(lambda data: {**data, 'text': ['My sentence: ' + i for i in data['text']]}),
                    batched = True,
                    batch_size = 1000,
                    num_proc = 4,
                    )
print(datasetTrain['text'][3])
print(maped_DatasetTrain2['text'][3])

房间太小。其他的都一般。。。。。。。。。
My sentence: 房间太小。其他的都一般。。。。。。。。。


Map (num_proc=4):   0%|          | 0/9600 [00:00<?, ? examples/s]

1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸，一摸一个印. 4.硬盘分区不好办.
My sentence: 1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸，一摸一个印. 4.硬盘分区不好办.


#### 13. 设置数据格式 利用 set_format() 函数
- type 表示数据修改的类别常用的值有 numpy, torch, tensorflow, pandas等
- columns 修改的字段
- output_all_columns 表示是否要保留其他字段

In [50]:
datasetTrain.set_format(type='torch', columns = ['label','text'], output_all_columns=True)
datasetTrain[20]

{'text': '非常不错，服务很好，位于市中心区，交通方便，不过价格也高！', 'label': tensor(1)}

#### 14.数据集保存为其他格式（csv,json）  还有其他多种格式如 sql、pandas、list、dict等
注意：对于数据集有多个split 时，需要逐个将进行 split 导出，函数不支持导入到一个文件中

In [69]:
# csv 格式
dataset = load_dataset(path = 'lansinuote/ChnSentiCorp', split='train')
dataset.to_csv('/root/ChnSentiCorp.csv')
csv_datasetTrain = load_dataset(path = 'csv',
                data_files = '/root/ChnSentiCorp.csv',
                split = 'train'
                                )
csv_datasetTrain[3]

Creating CSV from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

{'text': '1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸，一摸一个印. 4.硬盘分区不好办.',
 'label': 0}

In [70]:
# json 格式
dataset = load_dataset(path = 'lansinuote/ChnSentiCorp', split='train')
dataset.to_json('/root/ChnSentiCorp.json')
json_dataset = load_dataset(
    path = 'json',
    data_files = '/root/ChnSentiCorp.json',
    split = 'train',
)
json_dataset[3]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

{'text': '1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸，一摸一个印. 4.硬盘分区不好办.',
 'label': 0}